This idea of this notebook is to create a first (simple) DL model using all the same features used in the multinomial logit model.

Much inspiration was derived from https://towardsdatascience.com/use-machine-learning-to-predict-horse-racing-4f1111fb6ced.

In [166]:
import math
from importlib import reload
import deeplearninglib
reload(deeplearninglib)
from deeplearninglib import *

import wandb

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [167]:
# read in data

# select the same features used in the multinomial model
# X_columns = ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"]
X_columns = ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"]
# X_columns = ["mkt_prob"]
y_columns = ["win"]

# hyperparameters
learning_rate = 10e-3
epochs = 100

vacant_indicator = False

train_data_fn = "data\\runners_train.csv"
test_data_fn = "data\\runners_test.csv"

train_data = RacesDataset(train_data_fn, X_columns, y_columns, vacant_indicator=vacant_indicator)
test_data = RacesDataset(test_data_fn, X_columns, y_columns, vacant_indicator=vacant_indicator, scalar=train_data.scalar)

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [168]:
pd.options.display.max_columns = 1000 # was 20
train_data.races.iloc[:, train_data.X_columns].head()

age                                                              \
stall_number   1    2    3    4    5    6    7    8    9    10   11   12   13   
race_id                                                                         
11504         3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  0.0  0.0  0.0  0.0  0.0   
11505         4.0  6.0  9.0  7.0  5.0  8.0  5.0  5.0  5.0  4.0  5.0  0.0  0.0   
11506         3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  0.0  0.0  0.0   
11507         5.0  4.0  4.0  4.0  4.0  4.0  8.0  4.0  4.0  0.0  0.0  0.0  0.0   
11508         7.0  4.0  5.0  4.0  5.0  4.0  5.0  5.0  4.0  0.0  0.0  0.0  0.0   

                            blinkers                                          \
stall_number   14   15   16       1    2    3    4    5    6    7    8    9    
race_id                                                                        
11504         0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0      1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                cheekpieces                 \
stall_number   10   11   12   13   14   15   16          1    2    3    4    
race_id                                                                      
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  1.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  1.0  1.0  0.0   

                                                                          \
stall_number   5    6    7    8    9    10   11   12   13   14   15   16   
race_id                                                                    
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             daysLTO                                                          \
stall_number      1      2      3      4      5      6      7      8      9    
race_id                                                                        
11504          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0    0.0   
11505          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11506          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11507          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11508          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   

                                                    entire                 \
stall_number     10     11   12   13   14   15   16     1    2    3    4    
race_id                                                                     
11504           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11505         365.0  365.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11506         365.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11507           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11508           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   

                                                                          \
stall_number   5    6    7    8    9    10   11   12   13   14   15   16   
race_id                                                                    
1150

In [169]:
# build the neural network

output_layer_nodes = train_data.y.shape[1]
input_layer_nodes = train_data.X.shape[1]
bias = True

# torch.manual_seed(0)
model = net(input_layer_nodes, output_layer_nodes, bias=bias).to(device) # linear-relu-linear-softwax nn (1 hidden layer)
print(f"Model structure: {model}")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure: net(
  (neural_network): Sequential(
    (0): Linear(in_features=336, out_features=16, bias=True)
    (1): Sigmoid()
  )
)
Layer: neural_network.0.weight | Size: torch.Size([16, 336]) | Values : Parameter containing:
tensor([[ 0.0106, -0.0489, -0.0350,  ..., -0.0446, -0.0222,  0.0293],
        [ 0.0333,  0.0450,  0.0367,  ...,  0.0442, -0.0230,  0.0362],
        [-0.0373, -0.0492, -0.0199,  ..., -0.0095,  0.0107, -0.0178],
        ...,
        [ 0.0002,  0.0530,  0.0528,  ..., -0.0147, -0.0529, -0.0396],
        [-0.0075, -0.0136, -0.0219,  ...,  0.0174, -0.0451,  0.0359],
        [ 0.0298, -0.0054, -0.0197,  ..., -0.0069, -0.0440,  0.0146]],
       requires_grad=True) 

Layer: neural_network.0.bias | Size: torch.Size([16]) | Values : Parameter containing:
tensor([-0.0097,  0.0059,  0.0262, -0.0423,  0.0326, -0.0367,  0.0456, -0.0430,
        -0.0141, -0.0176,  0.0114,  0.0134, -0.0141,  0.0150, -0.0022,  0.0120],
       requires_grad=True) 



In [170]:
# example to show how model is used from prediction

X = torch.rand(1, input_layer_nodes, device=device)
logits = model(X)
y_pred = logits.argmax(1)
print(f"Predicted class: {y_pred}")
print(X.shape)

Predicted class: tensor([0])
torch.Size([1, 336])


In [171]:
%env "WANDB_NOTEBOOK_NAME" "3b_Deep Learning.ipynb"

# optimizing model parameters

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="horse-racing-project",
    
    # track hyperparameters and run metadata
    config={
    "device": device,
    "X_columns": X_columns,
    "learning_rate": learning_rate,
    "epochs": epochs,
    "vacant_indicator": vacant_indicator,
    "dataset": [train_data_fn, test_data_fn],
    "bias": bias,
    "architecture": list(model.modules())
    }
)

# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    (acc, loss) = test_loop(test_dataloader, model, loss_fn, device)
    wandb.log({"acc": acc, "loss": loss})
print("Done!")

# finish the wandb run, necessary in notebooks
wandb.finish()

env: "WANDB_NOTEBOOK_NAME"="3b_Deep Learning.ipynb"


Epoch 1
-------------------------------
loss: 2.818939  [   64/16620]
loss: 2.752923  [ 6464/16620]
loss: 2.799052  [12864/16620]
Test Error: 
 Accuracy: 10.6%, Avg loss: 2.744464 

Epoch 2
-------------------------------
loss: 2.802709  [   64/16620]
loss: 2.713400  [ 6464/16620]
loss: 2.762259  [12864/16620]
Test Error: 
 Accuracy: 12.0%, Avg loss: 2.715053 

Epoch 3
-------------------------------
loss: 2.788994  [   64/16620]
loss: 2.681955  [ 6464/16620]
loss: 2.734219  [12864/16620]
Test Error: 
 Accuracy: 13.3%, Avg loss: 2.692630 

Epoch 4
-------------------------------
loss: 2.777498  [   64/16620]
loss: 2.657082  [ 6464/16620]
loss: 2.712567  [12864/16620]
Test Error: 
 Accuracy: 13.9%, Avg loss: 2.675160 

Epoch 5
-------------------------------
loss: 2.767603  [   64/16620]
loss: 2.637084  [ 6464/16620]
loss: 2.695281  [12864/16620]
Test Error: 
 Accuracy: 14.5%, Avg loss: 2.661088 

Epoch 6
-------------------------------
loss: 2.758811  [   64/16620]
loss: 2.620655  [ 64

acc,▁▃▄▅▆▆▇▇▇▇▇▇▇███▇███████████████████████
loss,█▇▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.20828
loss,2.48911


In [172]:
for para_name, para_vals in model.named_parameters():
    np.savetxt(para_name + ".csv", para_vals.data.numpy(), fmt='%6.3f', delimiter=",")